# 用Python收邮件

### [认真对待 Python3 收邮件](https://blog.csdn.net/Marksinoberg/article/details/66969620)
### [python学习——POP3收取邮件](https://blog.csdn.net/youzhouliu/article/details/51952354)

#### 收取邮件就是编写一个MUA作为客户端，从MDA把邮件获取到用户的电脑或者手机上。收取邮件最常用的协议是POP协议，目前版本号是3，俗称POP3。
* Python内置一个poplib模块，实现了POP3协议，可以直接用来收邮件。
* 注意到POP3协议收取的不是一个已经可以阅读的邮件本身，而是邮件的原始文本，这和SMTP协议很像，SMTP发送的也是经过编码后的一大段文本。
* 要把POP3收取的文本变成可以阅读的邮件，还需要用email模块提供的各种类来解析原始文本，变成可阅读的邮件对象。
* 所以，收取邮件分两步：
** 第一步：用poplib把邮件的原始文本下载到本地；
** 第二部：用email解析原始文本，还原为邮件对象。

####    POP方法	                                                                                描述
* user(login)	向服务器发送登录名，并显示服务器的响应，表示服务器正在等待输入该用户的密
* pass_(passwd) 	在用户使用 user()登录后，发送 passwd。如果登录失败，则抛出异常 
* stat()	返回邮件的状态，即一个长度为 2 的元组（msg_ct, mbox_siz），分别表示消息的数量和消息的总大小（即 字节数） 
* list([msgnum]) 	stat()的扩展，从服务器返回以三元组表示的整个消息列表（rsp, msg_list, rsp_siz），分别为服务器的响应、 消息列表、返回消息的大小。如果给定了 msgnum，则只返回指定消息的数据 
* retr(msgnum)	从服务器中得到消息的 msgnum，并设置其“已读”标志。返回一个长度为 3 的元组（rsp, msglines, msgsiz）， 分别为服务器的响应、消息的 msgnum 的所有行、消息的字节数
* dele(msgnum) 	把消息 msgnum 标记为删除，大多数服务器在调用 quit()后执行删除操作 
* quit()	注销、提交修改（如处理“已读”和“删除”标记等）、解锁邮箱、终止连接，然后退出 

In [8]:
import poplib

# base64解码
import base64

#解码邮件信息
from email.parser import Parser

#解码邮件主题
from email.header import decode_header

#解析发件人详情，名称及地址
from email.utils import parseaddr

#输入邮件地址，口令和POP3服务器地址
email = "yanlaile1019@gmail.com"
password = "odtxybaggivhsmle" #客户端授权码而非登录密码，需要自己登陆去设置
#（http://www.91ymb.com/docs/articles/25/ru-he-she-zhi-shou-jian-you-xiang-he-shou-quan-ma）
pop3_server = "pop.gmail.com"

#### 返回邮箱中的最新邮件

In [35]:
def get_mail_content():
    useraccount = 'yanlaile1019@gmail.com'
    password = "odtxybaggivhsmle"
    pop3_server = 'pop.gmail.com
    
    #开始连接到服务器
    server = poplib.POP3_SSL(pop3_server)
    
    #打开或者关闭调试信息，为打开时，会在控制台打印客户端与服务器的交互信息
    server.set_debuglevel(1) # 1为打开
    
    #打印POP3服务器的欢迎文字，验证是否正确连接到了邮件服务器
    print(server.getwelcome().decode('utf8'))
    
    #开始进行身份验证
    server.user(useraccount)
    server.pass_(password)
    
    #返回邮件总数和占用服务器的空间大小（字节数），通过stat（）方法
    email_num, email_size = server.stat()
    print("消息的数量:{0},\n消息的总大小:{1}".format(email_num, email_size))
    
    #使用list()返回所有邮件的编号，默认为字节类型的串
    rsp, msg_list, rsp_size = server.list()
    print("服务器的响应:{0},\n消息列表:{1},\n返回消息的大小:{2}".format(rsp, msg_list, rsp_size))
    
    print("邮件总数: {}".format(len(msg_list)))
    
    #单纯获取最新一封的邮件
    total_mail_numbers = len(msg_list)
    rsg, msglines, msgsiz = server.retr(total_mail_numbers)
    
    #print("服务器的响应: {0},\n原始邮件内容: {1},\n改封邮件所占字节大小:{2}".format(rsp, msglines, msgsize))
    
    msg_content = b'\r\n'.join(msglines).decode('gbk')
    
    msg = Parser().parsestr(text=msg_content)
    print("解码后的邮件信息:\n{}".format(msg))
    
    #关闭服务器的连接，释放资源
    server.close()
    
    return msg




In [34]:
# import poplib
# import email
# import string

# mailServer = poplib.POP3_SSL("pop.gmail.com")
# mailServer.user('yanlaile1019@gmail.com')
# mailServer.pass_("odtxybaggivhsmle")
# (mailCount, size) = mailServer.stat()

# print("Mails: {0}".format(mailCount))

# for i in range(1, mailCount+1):
#     print("the No.{0} mail".format(i))
#     (hdr, messages, octet) = mailServer.retr(i)
#     #messages = [message.decode('utf-8') for message in messages]
#     mail = email.message_from_bytes('\n'.encode('utf-8').join(messages))
#     subject = email.header.decode_header(mail.get("subject"))
#     if type(subject[0][0]) in (type(b' '), ):
#                                     print("subject of the email:" + subject[0][0].decode(subject[0][1]))
#     else:
#                                     print("subject of the email:" + subject[0][0])
#     for par in mail.walk():
#                                     if not par.is_multipart():
#                                         #附件
#                                         name = par.get_param("name")
#                                         if name:
#                                             dh = email.header.decode_header(name)
#                                             if type(dh[0][0]) in (type(b' '), ):
#                                                 if dh[0][1] == None:
#                                                     fname = dh[0][0].decode()
#                                                 else:
#                                                     fname = dh[0][0].decode(dh[0][1])
#                                             else:
#                                                 fname = dh[0][0]
#                                             print('附件名：' + fname)
#                                             data = par.get_payload(decode=True)

#                                             try:
#                                                 f = open(fname, 'wb')
#                                             except:
#                                                 print('附件名有非法字符')
#                                             f.write(data)
#                                             f.close()
#                                         else:
#                                             #邮件内容
#                                             ch = par.get_content_charset()
#                                             if ch == None:
#                                                 print(par.get_payload(decode=True).decode())
#                                             else:
#                                                 print(par.get_payload(decode=True).decode(ch))

#     print("=================================")


### 解析邮件主题

In [41]:
# 用来解析邮件主题
from email.header import decode_header

def parser_subject(msg):
    subject = msg['Subject']
    value, charset = decode_header(subject)[0]
    if charset:
        value = value.decode(charset)
    print('邮件主题:{0}'.format(value))
    return value

### 解析邮件来源

In [18]:
# 用来解析邮件来源
from email.utils import parseaddr

def parser_address(msg):
    hdr,addr = parseaddr(msg['From'])
    # name 发件人邮箱名称， addr发件人邮箱地址
    name, charset = decode_header(hdr)[0]
    if charset:
        name = name.decode(charset)
        print('发送人邮箱名称:{0}, 发件人邮箱地址:{1}'.format(name, addr))

### 解析邮件内容
* 正文信息是被base64编码后的串，需要获取编码格式进行解码

In [43]:
def parser_content(msg):
    content = msg.get_payload()
    
    #文本信息
    content_charset = content[0].get_content_charset()#获取编码格式
    text = content[0].as_string().split('base64')[-1]
    text_content = base64.b64decode(text).decode(content_charset) # base64解码
    
    # 添加了HTML代码的信息
    content_charset = content[1].get_content_charset()
    text = content[1].as_sting().split('base64')[-1]
    html_content = base64.b64decode(text).decode(content_charset)
    
    print('文本信息：{0}\n添加了HTML代码的信息:{1}'.format(text_content, html_content))